In [2]:
#importing libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
# get the respone html for url 
linkurl='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html=requests.get(linkurl).text
# parse the response html using beautifulsoup xml parser
soup=BeautifulSoup(html,'lxml')

#find the table and load the relevant data
tabl=soup.find('table')


# ### Build the initial pandas dataframe
#Build a pandas dataframe 
data=[]
headers=[]

#parse through the table tags and obtain data and header values
for rowtext in tabl.find_all('tr'):
    if(headers==[]):
        columns =rowtext.find_all('th')
        if columns!=[]: 
            headers=[columns[0].text,columns[1].text,columns[2].text]
    else:
        columns =rowtext.find_all('td')
        
        if columns!=[]: 
            #build row values
            row=[columns[0].text.strip('\n'),columns[1].text.strip('\n'),columns[2].text.strip('\n')]
            data.append(row)
            
#Column names as given in the assignment
headers=['PostalCode', 'Borough', 'Neighborhood']

#Initial dataframe
df=pd.DataFrame(data,columns=headers)  
df.shape

(287, 3)

In [4]:


# ## Cleaning Data

# ### Removing rows wth unassigned Borough

# In[5]:


#If Borough value is Not assigned remove that row
assigned=df['Borough']!= 'Not assigned'
dfclean=df[assigned]
dfclean.shape

(210, 3)

In [5]:
# ### Replacing the unassigned neighborhood values with Borough value
for i in range(0,dfclean.shape[0]):
    if(dfclean.iloc[i]['Neighborhood']=='Not assigned'):
               dfclean.iloc[i]['Neighborhood']=dfclean.iloc[i]['Borough']
dfclean.shape

(210, 3)

In [6]:
# ### Merging the rows with same postalcode but different neoghborhood 
dfuniq=dfclean.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
dfuniq.shape

(103, 3)

In [7]:
df=dfuniq

In [8]:
#Read the csv file containing latitude and longiutde information
linkurl='https://cocl.us/Geospatial_data'
dflatlong=pd.read_csv(linkurl)
dflatlong.rename(columns={'Postal Code':'PostalCode'},inplace=True)
dflatlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# ### Merge the latitude and longitude data of the neighborhoods
#Merge the lat long data with the original dataframe
dffinal=pd.merge(df,dflatlong,on='PostalCode',how='inner')
dffinal.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
dffinal.shape

(103, 5)

In [11]:
df=dffinal

In [12]:
#install geopy and folium
!pip install geopy
!pip install folium

     |████████████████████████████████| 92kB 8.2MB/s eta 0:00:011


In [13]:
from geopy import Nominatim
import folium
import json
from pandas.io.json import json_normalize
from IPython.display import Image,HTML

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postal_code in zip(df['Latitude'], df['Longitude'], df['Borough'], df['PostalCode']):
    label = '{}, {}'.format(postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto.save('map_toronto.png')
    
map_toronto

In [16]:
#Include only the rows where Borough contains the word Toronto
isToronto =df['Borough'].str.find('Toronto')!=-1
toronto_data = df[isToronto]
print(toronto_data.shape)
toronto_data.head(5)

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
#Include only the rows where Borough contains the word Toronto
isToronto =df['Borough'].str.find('Toronto')!=-1
toronto_data = df[isToronto]
print(toronto_data.shape)
toronto_data.head(5)

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
#Get the FourSquare API credentials
CLIENT_ID = 'SUPHEQ3HQVETPPAXSAZFVA2B2JWRNORIQR2LAIV5J05OOIC2' # your Foursquare ID
CLIENT_SECRET = 'M1DT3BFHWUIWMNAXJTKH0GCSRAMJRLMK1NWLYI24MXTO0J4B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SUPHEQ3HQVETPPAXSAZFVA2B2JWRNORIQR2LAIV5J05OOIC2
CLIENT_SECRET:M1DT3BFHWUIWMNAXJTKH0GCSRAMJRLMK1NWLYI24MXTO0J4B


In [19]:
toronto_data.iloc[0]['Neighborhood']

'The Beaches'

In [20]:
#Get the geographical coordinates
neighborhood_latitude = toronto_data.iloc[0][ 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.iloc[0][ 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.iloc[0][ 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [21]:
#Build url to get the forst 100 venues
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SUPHEQ3HQVETPPAXSAZFVA2B2JWRNORIQR2LAIV5J05OOIC2&client_secret=M1DT3BFHWUIWMNAXJTKH0GCSRAMJRLMK1NWLYI24MXTO0J4B&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [22]:
# Make the call to FourSquare API and get the results(Regular call)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6b16b9211536001b76a8e0'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
import json
from pandas.io.json import json_normalize
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [32]:
cat=toronto_venues['Venue Category'].unique()
for categ in cat:
    print(categ)

Trail
Health Food Store
Pub
Neighborhood
Greek Restaurant
Cosmetics Shop
Italian Restaurant
Ice Cream Shop
Yoga Studio
Brewery
Fruit & Vegetable Store
Pizza Place
Bookstore
Restaurant
Dessert Shop
Juice Bar
Bubble Tea Shop
Spa
Diner
Grocery Store
Furniture / Home Store
Café
Coffee Shop
Bakery
Caribbean Restaurant
American Restaurant
Frozen Yogurt Shop
Lounge
Liquor Store
Gym
Fish & Chips Shop
Fast Food Restaurant
Sushi Restaurant
Park
Pet Store
Steakhouse
Burrito Place
Movie Theater
Sandwich Place
Fish Market
Gay Bar
Cheese Shop
Middle Eastern Restaurant
Comfort Food Restaurant
Thai Restaurant
Seafood Restaurant
Stationery Store
Coworking Space
Wine Bar
Bar
Gym / Fitness Center
Latin American Restaurant
Gastropub
Bank
Clothing Store
Convenience Store
Thrift / Vintage Store
Lake
Swim School
Bus Line
Food & Drink Shop
Breakfast Spot
Department Store
Hotel
Chinese Restaurant
Salon / Barbershop
Mexican Restaurant
Sporting Goods Shop
Shoe Store
Rental Car Location
Indian Restaurant
Toy / Ga

In [33]:

#Include only the rows where Category contains the word Restaurant
isRestaurant =toronto_venues['Venue Category'].str.find('Restaurant')!=-1
toronto_venues = toronto_venues[isRestaurant]
print(toronto_venues.shape)
toronto_venues.head(5)

(417, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
11,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
15,"The Danforth West, Riverdale",43.679557,-79.352188,7 Numbers,43.677062,-79.353934,Italian Restaurant


In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",31,31,31,31,31,31
Berczy Park,11,11,11,11,11,11
"Brockton, Exhibition Place, Parkdale Village",3,3,3,3,3,3
Business Reply Mail Processing Centre 969 Eastern,2,2,2,2,2,2
"Cabbagetown, St. James Town",11,11,11,11,11,11
Central Bay Street,24,24,24,24,24,24
"Chinatown, Grange Park, Kensington Market",29,29,29,29,29,29
Christie,2,2,2,2,2,2
Church and Wellesley,29,29,29,29,29,29


In [35]:
toronto_venues['Venue Category'].unique()

array(['Greek Restaurant', 'Italian Restaurant', 'Restaurant',
       'Caribbean Restaurant', 'American Restaurant',
       'Fast Food Restaurant', 'Sushi Restaurant',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Thai Restaurant', 'Seafood Restaurant',
       'Latin American Restaurant', 'Chinese Restaurant',
       'Mexican Restaurant', 'Indian Restaurant', 'Vietnamese Restaurant',
       'Japanese Restaurant', 'Taiwanese Restaurant', 'Theme Restaurant',
       'Ramen Restaurant', 'Ethiopian Restaurant',
       'Mediterranean Restaurant', 'Afghan Restaurant',
       'French Restaurant', 'Asian Restaurant',
       'Modern European Restaurant', 'New American Restaurant',
       'Vegetarian / Vegan Restaurant', 'German Restaurant',
       'Eastern European Restaurant', 'Portuguese Restaurant',
       'Falafel Restaurant', 'Korean Restaurant', 'Colombian Restaurant',
       'Brazilian Restaurant', 'Gluten-free Restaurant',
       'Belgian Restaurant', 'Dumpling R

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 46 uniques categories.


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_onehot.shape

(417, 47)

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide, King, Richmond",0.000000,0.064516,0.064516,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,...,0.000000,0.032258,0.161290,0.064516,0.096774,0.000000,0.129032,0.000000,0.064516,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.181818,0.181818,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.181818,0.000000,...,0.000000,0.000000,0.181818,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000
5,Central Bay Street,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.041667,0.041667,0.041667,0.041667,0.041667,0.000000,0.083333,0.000000,0.041667,0.000000
6,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.137931,0.000000,...,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103448,0.172414
7,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Church and Wellesley,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.034483,0.000000,...,0.000000,0.034483,0.103448,0.034483,0.103448,0.000000,0.034483,0.034483,0.000000,0.034483
9,"Commerce Court, Victoria Hotel",0.000000,0.133333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.233333,0.100000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000


In [40]:
toronto_grouped.shape

(32, 47)

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0                     Restaurant  0.16
1                Thai Restaurant  0.13
2               Sushi Restaurant  0.10
3               Asian Restaurant  0.06
4  Vegetarian / Vegan Restaurant  0.06


----Berczy Park----
                           venue  freq
0             Seafood Restaurant  0.18
1                     Restaurant  0.18
2               Greek Restaurant  0.09
3  Vegetarian / Vegan Restaurant  0.09
4            Japanese Restaurant  0.09


----Brockton, Exhibition Place, Parkdale Village----
                             venue  freq
0               Italian Restaurant  0.33
1              Japanese Restaurant  0.33
2                       Restaurant  0.33
3                Afghan Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Business Reply Mail Processing Centre 969 Eastern----
                             venue  freq
0             Fast Food Restaurant   0.5
1                       Restaurant

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Restaurant,Thai Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,American Restaurant,Asian Restaurant,Seafood Restaurant,Modern European Restaurant,Colombian Restaurant,Japanese Restaurant
1,Berczy Park,Restaurant,Seafood Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,Greek Restaurant,Eastern European Restaurant,Japanese Restaurant,French Restaurant,Thai Restaurant,Brazilian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Italian Restaurant,Japanese Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Cuban Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant
4,"Cabbagetown, St. James Town",Restaurant,Chinese Restaurant,Italian Restaurant,Thai Restaurant,Taiwanese Restaurant,Indian Restaurant,Caribbean Restaurant,Japanese Restaurant,Vietnamese Restaurant,Doner Restaurant


In [44]:
neighborhoods_venues_sorted.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Stn A PO Boxes 25 The Esplanade,Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Thai Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Eastern European Restaurant,Molecular Gastronomy Restaurant
28,Studio District,American Restaurant,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Seafood Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Dim Sum Restaurant,Falafel Restaurant,Ethiopian Restaurant
29,"The Annex, North Midtown, Yorkville",Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,American Restaurant,Indian Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
30,"The Beaches West, India Bazaar",Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
31,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,American Restaurant,Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Doner Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [45]:
neighborhoods=neighborhoods_venues_sorted['Neighborhood'].tolist()
hasdata=toronto_data.Neighborhood.isin(neighborhoods)
toronto_data =toronto_data[hasdata]
toronto_data.shape

(32, 5)

In [46]:
#Cluster the neighborhoods
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#isNan=toronto_merged['Cluster Labels'].notnull()
#toronto_merged=toronto_merged[isNan]
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,American Restaurant,Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Doner Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,American Restaurant,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Seafood Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Dim Sum Restaurant,Falafel Restaurant,Ethiopian Restaurant
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Fast Food Restaurant,Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Seafood Restaurant,Restaurant,Greek Restaurant,Indian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Ethiopian Restaurant


In [48]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,0,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
43,East Toronto,0,American Restaurant,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Seafood Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Dim Sum Restaurant,Falafel Restaurant,Ethiopian Restaurant
46,Central Toronto,0,Fast Food Restaurant,Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
47,Central Toronto,0,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Seafood Restaurant,Restaurant,Greek Restaurant,Indian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Ethiopian Restaurant
49,Central Toronto,0,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Restaurant,Dim Sum Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
51,Downtown Toronto,0,Restaurant,Chinese Restaurant,Italian Restaurant,Thai Restaurant,Taiwanese Restaurant,Indian Restaurant,Caribbean Restaurant,Japanese Restaurant,Vietnamese Restaurant,Doner Restaurant
52,Downtown Toronto,0,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant
53,Downtown Toronto,0,Mexican Restaurant,French Restaurant,Asian Restaurant,Restaurant,Dim Sum Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
54,Downtown Toronto,0,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Ramen Restaurant,Vietnamese Restaurant,Modern European Restaurant,Chinese Restaurant,Ethiopian Restaurant,Mexican Restaurant
55,Downtown Toronto,0,Restaurant,Italian Restaurant,American Restaurant,Thai Restaurant,Seafood Restaurant,Japanese Restaurant,New American Restaurant,Latin American Restaurant,Comfort Food Restaurant,Middle Eastern Restaurant


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,1,Restaurant,Italian Restaurant,Japanese Restaurant,Comfort Food Restaurant,Sushi Restaurant,Chinese Restaurant,French Restaurant,Doner Restaurant,Fast Food Restaurant,Falafel Restaurant
75,Downtown Toronto,1,Italian Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant
78,West Toronto,1,Italian Restaurant,Japanese Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2,Greek Restaurant,Italian Restaurant,American Restaurant,Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Doner Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,3,Sushi Restaurant,Vietnamese Restaurant,Cuban Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant


In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,West Toronto,4,Middle Eastern Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant
